In [1]:
infile_name = "20190911_8genre_288video_text"

!pip install japanize_matplotlib
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

# URLを開いて認証コードをコピーしボックスに貼り付けしてください
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
!cp -f ~/.gdfuse/default/config config
!sed -i -e "s/^root_folder=$/root_folder=1Dm6GSlI06zTsmRg6KdpM2WBYU0wj-Umh/" config

!mkdir -p drive
!google-drive-ocamlfuse -config ./config -cc drive
!ls drive

     |████████████████████████████████| 4.1MB 4.8MB/s 
  Created wheel for japanize-matplotlib: filename=japanize_matplotlib-1.0.5-cp36-none-any.whl size=4118721 sha256=6eb8c4159d49e132595f4cbbc9622b328086cadb4fb820533af50b52a265e2f6
  Stored in directory: /root/.cache/pip/wheels/6c/8a/08/4a784957da9f3c2b4839b4986be2fba2a481877318948be52c
Successfully built japanize-matplotlib
E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 132684 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.13-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.13-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.13-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleuserc

In [2]:
import os
import csv
import pickle

import numpy as np
import pandas as pd

import japanize_matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_formats = {'png', 'retina'}
sns.set(font='IPAexGothic')

from IPython.display import display, Image
import warnings
warnings.filterwarnings('ignore')
os.chdir("drive")
!ls

config	drive  dust  input  memo  midway  model  notebook  output


In [0]:
import sys
sys.path.append('/content/drive/notebook/NLP')
import vectorizer

In [0]:
#csvファイルをインポートして'labels'リストに格納
f = open(f"midway/objects_54files_191113.csv", "r")
reader = csv.reader(f)
visions = [e for e in reader]
f.close()
visions = visions[1:]

In [0]:
import re
import collections
corpus = collections.Counter()
vision_words = [[] for _ in visions]
n_nums = 0
n_words = 0
for i, v in enumerate(visions):
  v = v[1:]
  for w in np.unique(v):
    if w=='':
      continue
    if re.search('^[0-9]+(\.[0-9]+)?$', w)==None:
      corpus.update([w])
      vision_words[i].append(w)
      n_words += 1
    else:
      n_nums += 1

In [17]:
n_nums, n_words

(100313, 18444)

In [0]:
vec = vectorizer.SWEM()

In [0]:
data = vec.transform(vision_words)

In [0]:
import pandas as pd
df = pd.read_csv(f"midway/objects_54files_191113.csv")

In [15]:
df.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,...,30640,30641,30642,30643,30644,30645,30646,30647,30648,30649,30650,30651,30652,30653,30654,30655,30656,30657,30658,30659,30660,30661,30662,30663,30664,30665,30666,30667,30668,30669,30670,30671,30672,30673,30674,30675,30676,30677,30678,30679
0,007/syabekuri_20190107,Shabekuri 007,NaN,Neptune,Actor,Cream Stew,NaN,Interior Design Services,Nippon TV,Tutorial,Nippon Television Network System,1.2856500148773193,0.2295999974012375,0.19419999420642853,0.18960000574588776,0.18690000474452972,0.1858000010251999,0.16937342286109924,0.16590000689029694,0.15889999270439148,0.14480000734329224,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,007/syabekuri_20181126,Photo caption,Product,Meter,Flooring,NaN,H&M,Tuxedo,Photo caption,Gentleman,Salaryman,Tuxedo M.,Tuxedo,Photo caption,Salaryman,Gentleman,Tuxedo M.,Tuxedo,Socialite,Tuxedo M.,Public Relations,Television,Public,Tuxedo,Public Relations,Salaryman,Gentleman,Socialite,Tuxedo M.,Public,Tuxedo,Photo caption,Public Relations,Gentleman,Salaryman,Socialite,Tuxedo M.,Public,Photo caption,Product,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,bukatsu/bukatsu_02,Darkness,Art,Screenshot,Midnight Memories,NaN,Angle,Meter,Poster,Wallpaper,Graphics,Computer,M,Meter,Human,Screenshot,Phenomenon,Emotion,Bilibili,danmu,Up主,ACG,Screenshot,Japanese cartoon,Visual Effects,Subtitle,Song,Wallpaper,Hair M,Black hair,Screenshot,Black,Hair,Screenshot,Darkness,Portrait -m-,Hair M,Black hair,Close-up,Black,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,burari/burari_20181110,Nippon TV,Japan,Travel program,NaN,Travel,途中下車,Television,NaN,Broadcasting,Rerun,Neon sign,Neon,Art,Burari tochû gesha no tabi,Hollyhocks,Botanical garden,Botany,Flora,Annual plant,Rose family,Shrub,Garden,Rose,Tree,Botanical garden,Floristry,Flora,Botany,Garden,Shrub,Annual plant,Recreation,Tree,Plants,Floristry,Garden,Flora,Floral design,Shrub,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,aisyahenreki/aisya_20181117,Car,Product,Recreation,Recreation,Tree,Tree,Energy,Road,Mode of transport,Transport,Winter,Car,Sidewalk,Pedestrian,Street,Asphalt,Phenomenon,Tree,Recreation,Car,Street,Sidewalk,Pedestrian,Asphalt,Tree,Recreation,Gō Wakabayashi,Car,おぎやはぎの愛車遍歴 NO CAR,NO LIFE!,NaN,NaN,Actor,NaN,Photo caption,NaN,Gメン,Forehead,Photo caption,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
def load_vision(path):
  